In [1]:
# !pip install wheel
# !pip install h5py
# !pip install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [2]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import h5py
import yaml
import pickle
import os
import argparse
import random
from tqdm.notebook import tqdm
from collections import Counter
from dataclasses import dataclass, field
import contextlib
import math
from heapq import merge
from math import floor
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from itertools import product
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from statsmodels.stats.multitest import multipletests
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr

In [3]:
USE_YAM = False
if not USE_YAM:
    Y_ID = ''
    data = pd.read_csv('../processed_data/utrs.csv')
    data = data[['Gene Name', 'foreign']].dropna(axis=0).rename(columns={'Gene Name': 'gene', 'foreign':'seq'})
else:
    Y_ID = 'YAM'
    data = pd.read_csv('../processed_data/yamanishi_data.csv', header = 0)
    data = data.loc[data.utr_seq.notna()]
    data = data.loc[data.gene.notna()]
    data.drop_duplicates(inplace = True)
    data = data[['gene', 'utr_seq']].dropna(axis=0).rename(columns={'utr_seq':'seq'})

In [4]:
data.head()

,gene,seq
4,VPS8,ACATTTCTAAATATTTAATACAACTTTGGTTACATAAAAGTAAAAT...
5,SSA1,AGCCAATTGGTGCGGCAATTGATAATAACGAAAATGTCTTTTAATG...
6,ERP2,AGAACTTTTCAATCTACGAAAAATATATGTCCGCAATATAGAACAC...
7,FUN14,AGCAAGACAAATGACCAGATATAAACGAGGGTTATATTCTTTCGTT...
8,SPO7,AAAGAGTTGGAGGGCTTCTTCCTTCGAATAAGAGGTCATATTTACC...


In [5]:
KMER_SIZE = 6

In [9]:
motifs_df = pd.read_csv('../processed_data/found_motifs.csv')

In [11]:
set(y for x in motifs_df.motif for y in x)

{'A', 'C', 'G', 'K', 'M', 'N', 'R', 'S', 'T', 'Y'}

In [24]:
elem

'TTTGT'

In [25]:
replacements.get(elem, [elem])

['TTTGT']

In [34]:
elem = 'TAATNNNTAAT'

In [46]:
# Efficiency element
eff_el1 = "TATATA"
eff_el2 = "TTTTTATA"
eff_ctrl = "GCGCGC"
# Mutational scan of efficiency element?
# Positioning element
pos_el = "AAWAAA"
# Puf protein binding sites
puf1_2 = "TAATNNNTAAT"
puf3 = "TGTANATA"
puf4 = "TGTANANTA"
puf5 = "TGTANNNNTA"
puf6 = "TTGT"
# Poly-T sequences
poly_t = "TTTTTTTT"
_elems = list(set([eff_el1, eff_el2, eff_ctrl,
           pos_el,
           puf1_2, puf3, puf4, puf5, puf6,
           poly_t] + list(set(motifs_df.motif))))
specElements = []
# expand elements above by replacing Ns with A,T,G or C and Ws with A or T
replacements = {
    'N':['A', 'T', 'G', 'C'],
    'W': ['A', 'T'],
    'S': ['G', 'C'],
    'K': ['G', 'T'],
    'Y': ['C', 'T'],
    'R': ['A', 'G'],
    'M': ['A', 'C']
}

for elem in _elems:
    specElements.extend([''.join(y) for y in product(*(replacements.get(x, (x,)) for x in elem))])
# augment elements using all contiguous subsequences of size k - 1  of them, if the element size is larger than K
specElements = list(set(specElements + sum([[x[:-1], x[1:]] for x in specElements if len(x) > KMER_SIZE], [])))

In [47]:
data = data[data.seq.apply(lambda x: len(x)>=KMER_SIZE)]

In [48]:
len(specElements)

1200

### Compute Stride (overlap) of subsequence reading based on current subsequence entropy

In [49]:
def entropy(string):
    "Calculates the Shannon entropy of a string"

    # get probability of chars in string
    prob = [ float(string.count(c)) / len(string) for c in dict.fromkeys(list(string)) ]

    # calculate the entropy
    entropy = - sum([ p * math.log(p) / math.log(2.0) for p in prob ])

    return min(1, abs(entropy) / 2)
def compute_stride(seq):
    return max(1, round(len(seq) * (1 - entropy(seq))))

We apply stride computation to the 20 first specific elements above

In [50]:
search_strides = [compute_stride(s) for s in specElements[:20]]
list(map(tuple, zip(*[specElements,search_strides])))

[('TGTATACGT', 1),
 ('AATGGGTAAT', 2),
 ('CCCGGCTGG', 3),
 ('TGTATAGGT', 2),
 ('GTATATA', 2),
 ('TGTACATCT', 1),
 ('TAATTAGTAAT', 4),
 ('TGTAGTCTTA', 1),
 ('TGTACTTATA', 2),
 ('TAATCGATAAT', 2),
 ('TGTAAACGT', 1),
 ('TGTACCGGTA', 1),
 ('TAATTGATAAT', 4),
 ('CCGGGTGG', 3),
 ('TGTAATCCT', 1),
 ('GTAACCGTA', 1),
 ('GTAGTTGTA', 2),
 ('TGTAATACTA', 1),
 ('GTAGCTATA', 1),
 ('GTACTCTTA', 1)]

In [51]:
def search_kmer_adaptive(seq, sub, stride):
    # Search positions of subsequence in seq, while respecting provided stride, in case of a hit
    found = []
    pos = 0
    while pos < len(seq):
        if seq[pos:pos+len(sub)] == sub:
            found.append(pos)
            pos += stride
        else:
            pos += 1
    return found

def get_kmers_adaptive(seq, k, min_stride):
    #Get K-Mers existing in the sequence seq, while computing the stride based on entropy and 
    #respecting a minimum stride
    kmers = []
    pos = 0
    while pos < len(seq):
        if pos + k > len(seq):
            break
        kmers.append(seq[pos:pos+k])
        pos += max(min_stride, compute_stride(kmers[-1]))
    return kmers

def search_all_kmers(seq, specElements, kmer_size, min_stride=1):
    # sort specific elements based on their length, so in case of overlap hit,
    # found sequences are ordered in a logical matter
    # eg AGT and AGTAC found both in position j, the produced string will be "AGT AGTAC"
    specElements = sorted(specElements, key=len)
    pairs = []
    # Retrieve the found positions of each element and merge all the found elements positions together
    for elem in specElements:
        pos = search_kmer_adaptive(seq, elem, compute_stride(elem))
        new_words_len = [len(elem) for _ in range(len(pos))]
        new_words = [elem for _ in range(len(pos))]
        new_pairs = list(map(tuple, zip(*[pos,new_words_len,new_words]))) 
        pairs = merge(pairs, new_pairs)
    pairs = [(x[0],x[0] + x[1], x[2]) for x in list(pairs)]
    # For the remaining intervals, in between found elements, get the kmers of the specific kmer_size
    # and with the minimum stride min_stride
    final_sequence = []
    # start of sequence (before findings)
    if not pairs:
        return get_kmers_adaptive(seq, kmer_size, min_stride)
    final_sequence =  get_kmers_adaptive(seq[:pairs[0][0] + compute_stride(pairs[0][2])], kmer_size, min_stride)
    # middle of sequence (with intertwined findings)
    for cnt in range(len(pairs) - 1):
        final_sequence.append(pairs[cnt][2])
        final_sequence.extend(
            get_kmers_adaptive(
                seq[pairs[cnt][1] - compute_stride(pairs[cnt][2]) :
                    pairs[cnt+1][0] + compute_stride(pairs[cnt + 1][2])], kmer_size, min_stride))
    # end of sequence (after findings)
    final_sequence.append(pairs[-1][2])
    final_sequence.extend(
        get_kmers_adaptive(seq[pairs[-1][1] - compute_stride(pairs[-1][2]):],
                           kmer_size, min_stride))
    return final_sequence

### An example of how the first 3'UTR sequence is split based on the algorithm above

In [52]:
data.seq.iloc[0]

'ACATTTCTAAATATTTAATACAACTTTGGTTACATAAAAGTAAAATTTATACACCTCATTTCATTATGTAGATTCATATATAGAATACCAATTATGATTG'

In [53]:
search_all_kmers(data.seq.iloc[0], specElements, KMER_SIZE)

['ACATTT',
 'ATTTCT',
 'TTCTAA',
 'CTAAAT',
 'AAATAT',
 'AATATTTAAT',
 'ATACA',
 'CAACTT',
 'AACTTT',
 'CTTTGG',
 'TTGGTT',
 'ACATA',
 'ATAAA',
 'AAAAG',
 'AAGTAA',
 'GTAAAA',
 'AAAATT',
 'TATACA',
 'ATACA',
 'CACCTC',
 'CCTCAT',
 'TCATTT',
 'ATTTCA',
 'TTCATT',
 'CATTAT',
 'TGTAGAT',
 'TGTAGATT',
 'TTCATA',
 'ATATA',
 'ATATAT',
 'TATAT',
 'TATATA',
 'ATAGAA',
 'AGAATA',
 'AATACC',
 'TACCAA',
 'CCAATT',
 'CAATTA',
 'ATTATG',
 'TATGAT',
 'TGATTG']

In [54]:
from tqdm.notebook import tqdm
tqdm.pandas()

preprocessed_seq = data.seq.progress_apply(search_all_kmers, specElements=specElements, kmer_size=KMER_SIZE)
corpus = [y  for x in preprocessed_seq for y in x]

  0%|          | 0/4869 [00:00<?, ?it/s]

In [55]:
GCORPUS = f'../processed_data/.utrs_corpus_{KMER_SIZE}{Y_ID}'
with open(GCORPUS, 'w') as out:
    out.write('\n'.join([' '.join(c) for c in preprocessed_seq]))
!head ../processed_data/.utrs_corpus_$KMER_SIZE$Y_ID

head: cannot open '../processed_data/utrs_corpus_6' for reading: No such file or directory


In [56]:
GBUILD_DIR = 'glove/build'
VERBOSE = 2
VOCAB_FILE = f'../processed_data/.utrs_vocab_{KMER_SIZE}{Y_ID}'
MEMORY = 4
WINDOW_SIZE = 10
COOCCURRENCE_FILE = f'../processed_data/.utrs_coocc_{KMER_SIZE}_{WINDOW_SIZE}{Y_ID}.bin'
COOCCURRENCE_SHUF_FILE = f'../processed_data/.utrs_coocc_{KMER_SIZE}_{WINDOW_SIZE}{Y_ID}.shuf.bin'
SEED = 42
VECTOR_SIZE = 50
THREADS = 8
ETA = 0.05
X_MAX = 100
MAX_ITER = 50
SAVE_FILE = f'../processed_data/utrs_embeddings_{KMER_SIZE}_{WINDOW_SIZE}_{VECTOR_SIZE}{Y_ID}'

In [57]:
!$GBUILD_DIR/vocab_count -verbose $VERBOSE < $GCORPUS > $VOCAB_FILE

BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.200000 tokens.300000 tokens.400000 tokens.500000 tokens.600000 tokens.700000 tokens.Processed 742180 tokens.
Counted 5026 unique words.
Using vocabulary of size 5026.



In [58]:
import os
os.system(f"{GBUILD_DIR}/cooccur -memory {MEMORY} -vocab-file {VOCAB_FILE} -verbose {VERBOSE} -window-size {WINDOW_SIZE} < {GCORPUS} > {COOCCURRENCE_FILE}")

COUNTING COOCCURRENCES
window size: 10
context: symmetric
max product: 13752509
overflow length: 38028356
Reading vocab from file "../processed_data/.utrs_vocab_6"...loaded 5026 words.
Building lookup table...table contains 22112103 elements.
Processing token: 0100000200000300000400000500000600000700000Processed 742180 tokens.
Writing cooccurrences to disk.......2 files in total.
Merging cooccurrence files: processed 0 lines.100000 lines.200000 lines.300000 lines.400000 lines.500000 lines.600000 lines.700000 lines.800000 lines.900000 lines.1000000 lines.1100000 lines.1200000 lines.1300000 lines.1400000 lines.1500000 lines.1600000 lines.1700000 lines.1800000 lines.1900000 lines.2000000 lines.2100000 lines.2200000 lines.2300000 lines.2400000 lines.2500000 lines.2600000 lines.2700000 lines.2800000 lines.2900000 lines.3000000 lines.3100000 lines.3200000 lines.3300000 lines.3400000 lines.3500000 lines.3600000 lines.3700000 lines.3800000 lines.3900000 lines.4000000 lines.4100000 lines.420000

0

In [59]:
os.system(f"{GBUILD_DIR}/shuffle -memory {MEMORY} -verbose {VERBOSE} -seed {SEED} < {COOCCURRENCE_FILE} > {COOCCURRENCE_SHUF_FILE}")

Using random seed 42
SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 5690548 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.5690548 lines.Merging temp files: processed 5690548 lines.



0

In [60]:
os.system(f"{GBUILD_DIR}/glove -save-file {SAVE_FILE} -threads {THREADS} -input-file {COOCCURRENCE_SHUF_FILE} -eta {ETA} -x-max {X_MAX} -iter {MAX_ITER} -vector-size {VECTOR_SIZE} -vocab-file {VOCAB_FILE} -verbose {VERBOSE}")


TRAINING MODEL
Read 5690548 lines.
Initializing parameters...Using random seed 1639959197
done.
vector size: 50
vocab size: 5026
x_max: 100.000000
alpha: 0.750000
12/20/21 - 01:13.18AM, iter: 001, cost: 0.031083
12/20/21 - 01:13.19AM, iter: 002, cost: 0.027719
12/20/21 - 01:13.21AM, iter: 003, cost: 0.026443
12/20/21 - 01:13.22AM, iter: 004, cost: 0.024889
12/20/21 - 01:13.24AM, iter: 005, cost: 0.023428
12/20/21 - 01:13.25AM, iter: 006, cost: 0.021709
12/20/21 - 01:13.27AM, iter: 007, cost: 0.019350
12/20/21 - 01:13.28AM, iter: 008, cost: 0.016557
12/20/21 - 01:13.30AM, iter: 009, cost: 0.013950
12/20/21 - 01:13.31AM, iter: 010, cost: 0.011843
12/20/21 - 01:13.32AM, iter: 011, cost: 0.010250
12/20/21 - 01:13.33AM, iter: 012, cost: 0.009082
12/20/21 - 01:13.35AM, iter: 013, cost: 0.008229
12/20/21 - 01:13.36AM, iter: 014, cost: 0.007598
12/20/21 - 01:13.38AM, iter: 015, cost: 0.007120
12/20/21 - 01:13.39AM, iter: 016, cost: 0.006752
12/20/21 - 01:13.40AM, iter: 017, cost: 0.006463
12/2

0

In [61]:
mapping = pd.read_csv(SAVE_FILE + '.txt', sep=' ', index_col=0, header=None).apply(lambda x: np.array(x),axis=1).to_dict()
embedding_keys = [x for x in mapping]
embedding_mat = np.array([x for x in mapping.values()])
tokenized = [[embedding_keys.index(x) for x in seq] for seq in preprocessed_seq]

In [62]:
with open(SAVE_FILE + '.pkl', 'wb') as out:
    pickle.dump([data.gene.tolist(), tokenized, embedding_keys, embedding_mat], out)
    

In [63]:
import pickle
with open(f'../processed_data/utrs_glove_embeddings{Y_ID}.pkl', 'wb') as out:
    pickle.dump({'data': [np.mean([embedding_mat[i, :] for i in t],axis=0) for t in tokenized], 'gene': data.gene.tolist()} ,out)

In [64]:
SAVE_FILE

'../processed_data/utrs_embeddings_6_10_50'

In [65]:
with open(SAVE_FILE + '.pkl', 'rb') as inp:
    genes, tokenized, embedding_keys, embedding_mat = pickle.load(inp)

In [66]:
tok = tokenized[0]